In [31]:
from Acesso import Login
from Email import Email
from Moeda import Moeda
from RemoverArquivo import Remover
from Tempo import DataHora
import os
from glob import glob
from Query import Query
import pandas as pd

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

data=DataHora()

querys={

    'Pagar':

    """

    DECLARE @DTBASE DATETIME,@DTINICIO DATETIME,@DTFIM DATETIME,@DIAS SMALLINT

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DIAS=DAY(@DTBASE)

    SET @DTFIM=DATEADD(DAY,@DIAS*-1,@DTBASE)

    SET @DTINICIO=DATEADD(DAY,(DAY(DATEADD(MM,-1,@DTFIM))-1)*-1,DATEADD(MM,-1,@DTFIM))

    SELECT *
    FROM netfeira.vw_contapagar
    WHERE [Data de Pagamento] BETWEEN @DTINICIO AND @DTFIM AND [ID Situação]='LQ'


    """,

    'Contas':

    """
    
    SELECT * FROM netfeira.vw_planocontas
    ORDER BY 1    
        
    """

}

In [32]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [33]:
listagem=['CAIXA-CONSORCIO','FORNECEDORES','DISTRIBUICAO DE LUCRO','RECIFE','SALVADOR']

In [34]:
tabelas_df['Pagar'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'NFe', 'ID Fornecedor', 'Razão Social', 'Nome Fantasia', 'CNPJ',
       'ID Conta', 'Contas', 'ID Tipo Conta', 'Tipo de Conta',
       'ID Grupo Conta', 'Grupo', 'ID Situação', 'Situação do Título',
       'Título R$', 'Saldo R$', 'Valor Pago R$', 'Desconto R$', 'Total R$'],
      dtype='object')

In [35]:
tabelas_df['Contas'].columns

Index(['ID Contas', 'Plano de Contas'], dtype='object')

In [36]:
tabelas_df['Contas']['Plano de Contas']=tabelas_df['Contas']['Plano de Contas'].apply(lambda info: str(info).strip())

In [37]:
tabelas_df['Contas']=tabelas_df['Contas'].loc[~tabelas_df['Contas']['Plano de Contas'].isin(listagem)]

In [38]:
tabelas_df['Pagar']['Contas']=tabelas_df['Pagar']['Contas'].apply(lambda info: str(info).strip())

tabelas_df['Pagar']=tabelas_df['Pagar'].loc[~tabelas_df['Pagar']['Contas'].isin(listagem)]

In [39]:
tabelas_df['Pagar']['ID Mês']=tabelas_df['Pagar']['Data de Pagamento'].dt.month

In [40]:
tabelas_df['Pagar']['Mês']=tabelas_df['Pagar']['ID Mês'].apply(data.Mes)

In [41]:
tabelas_df['Pagar'].columns

Index(['Data de Emissão', 'Data de Vencimento', 'Data de Pagamento', 'Título',
       'NFe', 'ID Fornecedor', 'Razão Social', 'Nome Fantasia', 'CNPJ',
       'ID Conta', 'Contas', 'ID Tipo Conta', 'Tipo de Conta',
       'ID Grupo Conta', 'Grupo', 'ID Situação', 'Situação do Título',
       'Título R$', 'Saldo R$', 'Valor Pago R$', 'Desconto R$', 'Total R$',
       'ID Mês', 'Mês'],
      dtype='object')

In [42]:
pagos_df=pd.DataFrame()

pagos_df=tabelas_df['Pagar']

In [46]:
colunas=[l for l in pagos_df.columns.tolist() if l=='ID Conta']

In [47]:
temp_df=pd.DataFrame()

for coluna in colunas:
    
    df=pagos_df[[coluna,'ID Mês','Valor Pago R$']].groupby([coluna,'ID Mês'],as_index=False).sum()
    
    df.rename(columns={coluna:'ID Contas'},inplace=True)
    
    temp_df=pd.concat([temp_df,df],axis=0,ignore_index=True)
            
    #break
    
    pass

temp_df.sort_values('ID Contas',ascending=True,inplace=True)

In [48]:
temp_df['Mês']=temp_df['ID Mês'].apply(data.Mes)

temp_df['Mês']=temp_df.apply(lambda info: str(info['ID Mês'])+' - '+info['Mês'],axis=1)

In [49]:
if(temp_df['ID Mês'].max()==12):
    
    temp_df.sort_values('Mês',ascending=False,inplace=True)
    
    pass

else:
    
    temp_df.sort_values('Mês',ascending=True,inplace=True)
    
    pass

In [50]:
temp_df=temp_df[['ID Contas','Mês','Valor Pago R$']].pivot(index='ID Contas',columns='Mês',values='Valor Pago R$').reset_index()

In [51]:
tabelas_df['Contas']=tabelas_df['Contas'].merge(temp_df,on='ID Contas',how='inner')

In [52]:
tabelas_df['Contas'].loc[tabelas_df['Contas'].iloc[:,-1].isnull(),tabelas_df['Contas'].columns[-1]]=0

In [53]:
tabelas_df['Contas'].loc[tabelas_df['Contas'].iloc[:,-2].isnull(),tabelas_df['Contas'].columns[-2]]=0

In [54]:
#tabelas_df['Contas']=tabelas_df['Contas'].loc[~(tabelas_df['Contas'][tabelas_df['Contas'].columns[-2]]==0)|~(tabelas_df['Contas'][tabelas_df['Contas'].columns[-1]]==0)]

In [55]:
tabelas_df['Contas'].loc[tabelas_df['Contas'].iloc[:,-1]==0,'Rep %']=0

In [56]:
tabelas_df['Contas'].loc[tabelas_df['Contas'].iloc[:,-2]==0,'Rep %']=0

In [57]:
tabelas_df['Contas'].loc[tabelas_df['Contas']['Rep %'].isnull(),'Rep %']=tabelas_df['Contas'].loc[tabelas_df['Contas']['Rep %'].isnull()].apply(lambda info: round((info[-2]/info[-3])-1,4)*100 if info[-2]>0 and info[-3]>0 else 0,axis=1)

In [58]:
tabelas_df['Contas']

,ID Contas,Plano de Contas,6 - JUNHO,7 - JULHO,Rep %
0,1.1.1,ALUGUEL,29758.70,35439.30,19.09
1,1.1.10,MATERIAL DE ESCRITORIO,3334.41,3191.56,-4.28
2,1.1.12,MATERIAL DE LIMPEZA,1925.77,2052.81,6.60
3,1.1.13,SEGURO DE VIDA,0.00,2613.20,0.00
4,1.1.14,TARIFAS BANCARIAS,15803.73,11921.54,-24.57
...,...,...,...,...,...
60,2.6.8,IRPJ,20546.57,19063.37,-7.22
61,2.6.9,FUNRURAL,522.75,191.25,-63.41
62,4.2.3,MAQUINAS E EQUIPAMENTOS,4213.08,2531.07,-39.92
63,4.2.4,MAQ E EQUI P/ ALUGUEL E COMODA,2733.44,6509.83,138.16


In [59]:
tabelas_df['Contas']['Diferença R$']=tabelas_df['Contas'].iloc[:,-2]-tabelas_df['Contas'].iloc[:,-3]

In [60]:
tabelas_df['Contas'].loc[tabelas_df['Contas']['Rep %'].isnull(),'Rep %']=0

In [61]:
tabelas_df['Contas']

,ID Contas,Plano de Contas,6 - JUNHO,7 - JULHO,Rep %,Diferença R$
0,1.1.1,ALUGUEL,29758.70,35439.30,19.09,5680.60
1,1.1.10,MATERIAL DE ESCRITORIO,3334.41,3191.56,-4.28,-142.85
2,1.1.12,MATERIAL DE LIMPEZA,1925.77,2052.81,6.60,127.04
3,1.1.13,SEGURO DE VIDA,0.00,2613.20,0.00,2613.20
4,1.1.14,TARIFAS BANCARIAS,15803.73,11921.54,-24.57,-3882.19
...,...,...,...,...,...,...
60,2.6.8,IRPJ,20546.57,19063.37,-7.22,-1483.20
61,2.6.9,FUNRURAL,522.75,191.25,-63.41,-331.50
62,4.2.3,MAQUINAS E EQUIPAMENTOS,4213.08,2531.07,-39.92,-1682.01
63,4.2.4,MAQ E EQUI P/ ALUGUEL E COMODA,2733.44,6509.83,138.16,3776.39
